## Network creation

Using the transformed data from the previous step, create networks for February and April.

In [1]:
# to allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)

import networkx as nx
import pickle

from data_processing.data_paths import OUT, GRAPHS
from model.network_data import NetworkData
from model.mobility_network import MobilityNetwork

Define a seed for random generators.

In [2]:
SEED = 4401

Read transformed data from pickled files.

In [3]:
demographics = pickle.load(open(f'{OUT}demographics.pkl', 'rb'))

comb_counts_pre = pickle.load(open(f'{OUT}comb_counts_pre.pkl', 'rb'))
comb_counts_post = pickle.load(open(f'{OUT}comb_counts_post.pkl', 'rb'))

trip_counts_pre = pickle.load(open(f'{OUT}trip_counts_pre.pkl', 'rb'))
trip_counts_post = pickle.load(open(f'{OUT}trip_counts_post.pkl', 'rb'))

Create network data instances.

In [4]:
network_data_pre = NetworkData(demographics, comb_counts_pre, trip_counts_pre)
network_data_post = NetworkData(demographics, comb_counts_post, trip_counts_post)

trip_count_change = NetworkData.calc_trip_count_change(network_data_pre, network_data_post)

Calculate the adjacency lists and cumulative probabilities.

In [5]:
network_data_pre.create_adjacency_list()
network_data_post.create_adjacency_list()

network_data_pre.create_cum_prob()
network_data_post.create_cum_prob()

Create the networks.

In [6]:
%%time
N = 10000
BASELINE = 3

network_pre = MobilityNetwork(network_data_pre, N=N, baseline=BASELINE,
                              multiplier=False,
                              trip_count_change=trip_count_change,
                              seed=SEED)

network_post = MobilityNetwork(network_data_post, N=N, baseline=BASELINE,
                               multiplier=True,
                               trip_count_change=trip_count_change,
                               seed=SEED)

network_pre.create()
network_post.create()

g_pre = network_pre.g
g_post = network_post.g

Breaking up pairs... 28906 of 28908
Breaking up pairs...
Breaking up pairs... 14304 of 14306
Breaking up pairs...
CPU times: user 1min 8s, sys: 1.55 s, total: 1min 10s
Wall time: 1min 9s


Save the graphs to graphml format.

In [7]:
nx.write_graphml(g_pre, f'{GRAPHS}pre_N_10000.graphml')
nx.write_graphml(g_post, f'{GRAPHS}post_N_10000.graphml')